# calculate the reference values for the 4 outputs

In [63]:
import numpy as np
from glob import glob
from skimage.morphology import skeletonize_3d
from medutils.medutils import load_itk, save_itk
import SimpleITK as sitk
import itk
from tqdm import tqdm
import matplotlib.pyplot as plt
from skimage.morphology import disk,diamond,rectangle,square,erosion,dilation,opening,closing,skeletonize
import pickle
import pandas as pd
import json
import matplotlib
import copy
import pingouin as pg

from scipy.stats import pearsonr


In [7]:
file_fpath = 'all_biomarkers_correct3.csv'
df=pd.read_csv(file_fpath)

In [57]:
net_ts_pred_fpath = "/home/jjia/data/lung_function/lung_function/scripts/results/experiments/2426/test_pred.csv"
df_net_ts_pred = pd.read_csv(net_ts_pred_fpath)

In [58]:
# remove one patient without the % predicted values because of the insuffient technique
pat_id_wo_pred = 8334199
df_net_ts_pred = df_net_ts_pred[df_net_ts_pred['pat_id']!=pat_id_wo_pred]


In [59]:
# extract testing dataset form the whole dataset
first_ts_pat_id = 7957098
test_df = df.loc[df['PatID'] >= first_ts_pat_id]


In [70]:
# icc
def icc(label, pred, columns = []):
    label['ID'] = np.arange(1, len(label) + 1)
    label['rater'] = 'label'

    pred['ID'] = np.arange(1, len(pred) + 1)
    pred['rater'] = 'pred'

    data = pd.concat([label, pred], axis=0)

    for column in columns:
        icc = pg.intraclass_corr(data=data, targets='ID', raters='rater', ratings=column).round(2)
        icc = icc.set_index("Type")
        icc = icc.loc['ICC2']['ICC']
        print(f"icc for {column}: {icc}")
        # prefix = label_fpath.split("/")[-1].split("_")[0]
        # icc_dict['icc_' + prefix + '_' + column] = icc

In [69]:
# show the r between net output and pft absolute values
for net_out_name, pft_abs_name in zip(['DLCOc_SB', 'FEV1', 'FVC', 'TLC_He'],['DLCOc', 'FEV1', 'FVC', 'TLC']):
    corr, p_value = pearsonr(df_net_ts_pred[net_out_name], test_df[pft_abs_name])
    mean = (df_net_ts_pred[net_out_name] - test_df[pft_abs_name]).mean()
    std = (df_net_ts_pred[net_out_name] - test_df[pft_abs_name]).std()
    print(f"{pft_abs_name}:\t r: {corr:.3f}, mean: {mean:.2f}, std, {std:.2f}")


DLCOc:	 r: 0.614, mean: -0.61, std, 1.70
FEV1:	 r: 0.764, mean: 0.01, std, 0.54
FVC:	 r: 0.828, mean: -0.46, std, 0.60
TLC:	 r: 0.823, mean: -0.72, std, 0.79


In [61]:
assert len(df_net_ts_pred) == len(test_df)
df_net_ts_pred = df_net_ts_pred.set_axis(range(len(df_net_ts_pred)))
test_df = test_df.set_axis(range(len(test_df)))

In [62]:
# show the r between net output and pft absolute values
for net_out_name, pft_abs_name in zip(['DLCOc_SB', 'FEV1', 'FVC', 'TLC_He'],['DLCOc', 'FEV1', 'FVC', 'TLC']):
    corr, p_value = pearsonr(df_net_ts_pred[net_out_name]/test_df[pft_abs_name+"Ref"], test_df[pft_abs_name]/test_df[pft_abs_name+"Ref"])
    print(pft_abs_name + 'PP', corr)

DLCOcPP 0.24098050709944877
FEV1PP 0.39002495986930613
FVCPP 0.5448626733817346
TLCPP 0.6603650688872379
